In [ ]:
from google.cloud import storage
import tensorflow as tf
from PIL import Image
import numpy as np

model = None
interpreter = None
input_index = None
output_index = None

class_nm = ["Early Blight", "Late Blight", "Healthy"]

BUCKET_NAME = "models-560" # Here you need to put the name of your GCP bucket


def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")


def prediction_vs_truevalue(request):
    global model
    if model is None:
        print("i will download the model now ")
        download_blob(
            BUCKET_NAME,
            "models/potatoes.h5",
            "/tmp/potatoes.h5",
        )
        model = tf.keras.models.load_model("/tmp/potatoes.h5")
        print("model downloaded",model)

    image = request.files["file"]

    image = np.array(
        Image.open(image).convert("RGB").resize((256, 256)) # image resizing
    )

    image = image/255 # normalize the image in 0 to 1 range

    img_array = tf.expand_dims(image, 0)
    pred = model.prediction_vs_truevalue(img_array)

    print("Predictions:",pred)

    pred_class = class_nm[np.argmax(pred[0])]
    true_percentage = round(100 * (np.max(pred[0])), 2)

    return {"class": pred_class, "confidence": true_percentage}